In [1]:
from bs4 import BeautifulSoup
import requests


# wikipedia 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
# check for status code and headers


#url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
result = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

print(result.status_code)
print(result.headers)


200
{'Date': 'Tue, 11 Jun 2019 13:07:43 GMT', 'Content-Type': 'text/html; charset=UTF-8', 'Content-Length': '14902', 'Connection': 'keep-alive', 'Server': 'mw1254.eqiad.wmnet', 'X-Content-Type-Options': 'nosniff', 'P3P': 'CP="This is not a P3P policy! See https://en.wikipedia.org/wiki/Special:CentralAutoLogin/P3P for more info."', 'X-Powered-By': 'HHVM/3.18.6-dev', 'Content-language': 'en', 'Last-Modified': 'Thu, 06 Jun 2019 10:22:24 GMT', 'Backend-Timing': 'D=108335 t=1560103974168521', 'Content-Encoding': 'gzip', 'Vary': 'Accept-Encoding,Cookie,Authorization,X-Seven', 'X-Varnish': '595197947 275277105, 534572633 914171420', 'Via': '1.1 varnish (Varnish/5.1), 1.1 varnish (Varnish/5.1)', 'Age': '68006', 'X-Cache': 'cp1081 hit/4, cp1085 hit/17', 'X-Cache-Status': 'hit-front', 'Server-Timing': 'cache;desc="hit-front"', 'Strict-Transport-Security': 'max-age=106384710; includeSubDomains; preload', 'Set-Cookie': 'WMF-Last-Access=11-Jun-2019;Path=/;HttpOnly;secure;Expires=Sat, 13 Jul 2019 12

Clean the data

In [2]:
soup = BeautifulSoup(result.content, 'html.parser')
table = soup.find('table')
trs = table.find_all('tr')
rows = []
for tr in trs:
    i = tr.find_all('td')
    if i:
        rows.append(i)
        
lst = []
for row in rows:
    postalcode = row[0].text.rstrip()
    borough = row[1].text.rstrip()
    neighborhood = row[2].text.rstrip()
    if borough != 'Not assigned':
        if neighborhood == 'Not assigned':
            neighborhood = borough
        lst.append([postalcode, borough, neighborhood])

lst

[['M3A', 'North York', 'Parkwoods'],
 ['M4A', 'North York', 'Victoria Village'],
 ['M5A', 'Downtown Toronto', 'Harbourfront'],
 ['M5A', 'Downtown Toronto', 'Regent Park'],
 ['M6A', 'North York', 'Lawrence Heights'],
 ['M6A', 'North York', 'Lawrence Manor'],
 ['M7A', "Queen's Park", "Queen's Park"],
 ['M9A', 'Etobicoke', 'Islington Avenue'],
 ['M1B', 'Scarborough', 'Rouge'],
 ['M1B', 'Scarborough', 'Malvern'],
 ['M3B', 'North York', 'Don Mills North'],
 ['M4B', 'East York', 'Woodbine Gardens'],
 ['M4B', 'East York', 'Parkview Hill'],
 ['M5B', 'Downtown Toronto', 'Ryerson'],
 ['M5B', 'Downtown Toronto', 'Garden District'],
 ['M6B', 'North York', 'Glencairn'],
 ['M9B', 'Etobicoke', 'Cloverdale'],
 ['M9B', 'Etobicoke', 'Islington'],
 ['M9B', 'Etobicoke', 'Martin Grove'],
 ['M9B', 'Etobicoke', 'Princess Gardens'],
 ['M9B', 'Etobicoke', 'West Deane Park'],
 ['M1C', 'Scarborough', 'Highland Creek'],
 ['M1C', 'Scarborough', 'Rouge Hill'],
 ['M1C', 'Scarborough', 'Port Union'],
 ['M3C', 'North 

In [3]:
import numpy as np
import pandas as pd

In [4]:
df = pd.DataFrame(lst, columns= ['postalcode', 'borough', 'neighborhood'])
print(df.shape)

(211, 3)


Delete rows with "Not assigned"

In [5]:
df_final = df[df.borough != 'Not assigned']
df_final.reset_index(drop = True, inplace = True)

df_final = df.groupby(['postalcode', 'borough'], as_index=False).agg(lambda neighborhoods: ', '.join(neighborhoods))

In [6]:
df_final.head()

,postalcode,borough,neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [7]:
df_final.shape

(103, 3)

Add columns Latitude & Longitude in df_final

In [8]:
df_final['Latitude'] = 0.00
df_final['Longitude'] = 0.00

In [9]:
df_final.shape

(103, 5)

Get the csv file with the geolocation data

In [10]:
!wget -O Geospatial_Coordinates.csv http://cocl.us/Geospatial_data/

--2019-06-11 09:11:49--  http://cocl.us/Geospatial_data/
Resolving cocl.us... 169.48.113.201
Connecting to cocl.us|169.48.113.201|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data/ [following]
--2019-06-11 09:11:49--  https://cocl.us/Geospatial_data/
Connecting to cocl.us|169.48.113.201|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-06-11 09:11:50--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com... 107.152.27.197, 107.152.26.197
Connecting to ibm.box.com|107.152.27.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-06-11 09:11:50--  https://ibm.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Reusing existing conne

Read the data

In [11]:
df_geocord = pd.read_csv('Geospatial_Coordinates.csv')
df_geocord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Pull geolocation data from df_geoform and add to df_final

In [12]:
for idx in df_final.index:
    geocord_idx = df_geocord['Postal Code'] == df_final.loc[idx, 'postalcode']
    df_final.at[idx, 'Latitude'] = df_geocord.loc[geocord_idx, 'Latitude'].values
    df_final.at[idx, 'Longitude'] = df_geocord.loc[geocord_idx, 'Longitude'].values


df_final.head()

,postalcode,borough,neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [13]:
df_final.shape

(103, 5)